Taller 1: Britney Yuliana Madroñero Lanz y Julián Andrey Rodas Suárez

In [1]:
#librerias
import pandas as pd
import numpy as np

In [2]:
#datos de la GEIH 
df_cgsse= pd.read_csv( "https://raw.githubusercontent.com/niconomist98/DataAnalyticsUQ/refs/heads/main/Datos/GEIH2025/cgsse.CSV"
                      ,encoding="latin",sep=";")

df_desocupados= pd.read_csv( "https://raw.githubusercontent.com/niconomist98/DataAnalyticsUQ/refs/heads/main/Datos/GEIH2025/No%20ocupados.CSV"
                      ,encoding="latin",sep=";")    

df_ocupados= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Ocupados.CSV"
                      ,encoding="latin",sep=";")

df_ft= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Fuerza%20de%20trabajo.CSV"
                      ,encoding="latin",sep=";")  

/tmp/ipykernel_4695/2176691736.py:8: DtypeWarning: Columns (28,89,133,147,158) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ocupados= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Ocupados.CSV"
/tmp/ipykernel_4695/2176691736.py:11: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ft= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Fuerza%20de%20trabajo.CSV"


1. Para el total nacional calcule :

-población total

In [3]:
ptc = df_cgsse["FEX_C18"].sum()
print('Población total:', int(ptc))

Población total: 52045001


-población en edad de trabajar

In [4]:
et = df_cgsse[df_cgsse['P6040']>=15]['FEX_C18'].sum()
print('Poblacion total en edad de trabjar:',int(et))

Poblacion total en edad de trabjar: 40653416


-población que no está en edad de trabajar

In [5]:
set = df_cgsse[df_cgsse['P6040']<15]['FEX_C18'].sum()
print('Poblacion total sin edad de trabajar:',int(set))

Poblacion total sin edad de trabajar: 11391585


-población económicamente activa

In [6]:
pea = df_ocupados["FEX_C18"].sum() + df_desocupados[df_desocupados['DSI']==1]["FEX_C18"].sum()
print(f'Poblacion economicamente activa:', int(pea))

Poblacion economicamente activa: 25974846


-población económicamente inactiva

In [7]:
pei = df_ft[df_ft['FFT']==1]['FEX_C18'].sum()
print(f'Poblacion economicamente inactiva:', int(pei))


Poblacion economicamente inactiva: 14678569


-tasa de ocupación

In [8]:
TO = (df_ocupados['FEX_C18'].sum() / et)

print(f'La tasa de ocupacion: {TO:.2%}')

La tasa de ocupacion: 58.43%


-tasa de desempleo


In [9]:
TD = (df_desocupados[df_desocupados['DSI']==1]['FEX_C18'].sum() / pea)
print(f'La tasa de desempleo: {TD:.2%}')

La tasa de desempleo: 8.55%


2. Calcule las tasas de desempleo para hombres y mujeres en todos los departamentos de colombia, cuales son los departamentos con mayores y menores tasas de desempleo ? 

In [11]:
df_desocupados_gen = pd.merge(df_desocupados[df_desocupados["DSI"]==1], df_cgsse[['DIRECTORIO','SECUENCIA_P','ORDEN','P3271']], on=['DIRECTORIO','SECUENCIA_P','ORDEN'], how='left')

desocupados_sexo = (df_desocupados_gen.groupby(['DPTO','P3271'])['FEX_C18'].sum().reset_index().rename(columns={'FEX_C18':'desocupados'}))

df_ocupados_gen = pd.merge(df_ocupados, df_cgsse[['DIRECTORIO','SECUENCIA_P','ORDEN','P3271']], on=['DIRECTORIO','SECUENCIA_P','ORDEN'], how='left')

ocupados_sexo = (df_ocupados_gen.groupby(['DPTO','P3271'])['FEX_C18'].sum().reset_index().rename(columns={'FEX_C18':'ocupados'}))

#ocupados y desocupados para calcular la PEA
pea_sexo = pd.merge(ocupados_sexo, desocupados_sexo, on=['DPTO','P3271'], how='outer').fillna(0)
pea_sexo['PEA'] = pea_sexo['ocupados'] + pea_sexo['desocupados']

#tasa de desempleo (TD)
pea_sexo['TD'] = pea_sexo['desocupados'] / pea_sexo['PEA'] * 100

pea_sexo['Sexo'] = pea_sexo['P3271'].map({1:'Hombres', 2:'Mujeres'})

print(pea_sexo[['DPTO','Sexo','TD']].head(20))

    DPTO     Sexo         TD
0      5  Hombres   4.886691
1      5  Mujeres   9.770415
2      8  Hombres   6.301949
3      8  Mujeres  10.232567
4     11  Hombres   9.821256
5     11  Mujeres   7.078481
6     13  Hombres   4.791098
7     13  Mujeres  12.028807
8     15  Hombres   6.967972
9     15  Mujeres   4.008697
10    17  Hombres   5.583183
11    17  Mujeres   7.677962
12    18  Hombres   2.676220
13    18  Mujeres  15.726909
14    19  Hombres   3.390661
15    19  Mujeres   5.146277
16    20  Hombres   5.991357
17    20  Mujeres  13.209651
18    23  Hombres   5.832048
19    23  Mujeres  19.837285


In [12]:
# Departamentos con mayor y menor tasa de desempleo en hombres y mujeres
td_max = pea_sexo.loc[pea_sexo.groupby("Sexo")["TD"].idxmax()]
td_min = pea_sexo.loc[pea_sexo.groupby("Sexo")["TD"].idxmin()]

print("Departamentos con MAYOR tasa de desempleo para hombres y mujeres:")
print(td_max[['DPTO','Sexo','TD']])

print("\nDepartamentos con MENOR tasa de desempleo para hombres y mujeres:")
print(td_min[['DPTO','Sexo','TD']])

Departamentos con MAYOR tasa de desempleo para hombres y mujeres:
    DPTO     Sexo         TD
48    81  Hombres  18.057931
49    81  Mujeres  31.660892

Departamentos con MENOR tasa de desempleo para hombres y mujeres:
    DPTO     Sexo        TD
56    91  Hombres  1.979606
57    91  Mujeres  1.874288


De acuerdo a los resultados podemos observar como Arauca(81), se posiciona como el departamento con mayor tasa de desempleo para hombres y mujeres en Colombia, por el contrario se evidencia que el Amazonas(91) es el departamento con la menor tasa.

3.Tomando en consideración únicamente los siguientes grupos de edad: (18 a 24), (24 a 30), (30 a 40), (40 a 50), (60 en adelante).

Para cada uno de los departamentos, cual es el grupo de edad con mayor y menor tasa de desempleo ? 

In [14]:
def clasificar_edad(x):
    if 18 <= x < 24:
        return "18-24"
    elif 24 <= x < 30:
        return "24-30"
    elif 30 <= x < 40:
        return "30-40"
    elif 40 <= x < 50:
        return "40-50"
    elif x >= 60:
        return "60+"
    else:
        return "Otros"

In [16]:
df_cgsse["grupo_edad"] = df_cgsse["P6040"].apply(clasificar_edad)

#desocupados y ocupados por grupo de edad

df_desocupados_gen = pd.merge(df_desocupados[df_desocupados["DSI"]==1], df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","grupo_edad"]], on=["DIRECTORIO","SECUENCIA_P","ORDEN"], how="left")

desocupados_edad = (df_desocupados_gen.groupby(["DPTO","grupo_edad"])["FEX_C18"].sum().reset_index().rename(columns={"FEX_C18":"desocupados"}))

df_ocupados_gen = pd.merge(df_ocupados, df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","grupo_edad"]], on=["DIRECTORIO","SECUENCIA_P","ORDEN"], how="left")

ocupados_edad = (df_ocupados_gen.groupby(["DPTO","grupo_edad"])["FEX_C18"].sum().reset_index().rename(columns={"FEX_C18":"ocupados"}))

#poblacion economicamente activa y tasa de desempleo por grupos de edad

pea_edad = pd.merge(ocupados_edad, desocupados_edad, on=["DPTO","grupo_edad"], how="outer").fillna(0)
pea_edad["PEA"] = pea_edad["ocupados"] + pea_edad["desocupados"]
pea_edad["TD"] = pea_edad["desocupados"] / pea_edad["PEA"] * 100

In [19]:
# Filtrar 
pea_edad = pea_edad[pea_edad["grupo_edad"].isin(["18-24","24-30","30-40","40-50","60+"])] 

mayores = pea_edad.loc[pea_edad.groupby("DPTO")["TD"].idxmax()]
menores = pea_edad.loc[pea_edad.groupby("DPTO")["TD"].idxmin()]

print("Grupo de edad con mayor tasa de desempleo por departamento:")
print(mayores[["DPTO","grupo_edad","TD"]])

print("Grupo de edad con menor tasa de desempleo por departamento:")
print(menores[["DPTO","grupo_edad","TD"]])

Grupo de edad con mayor tasa de desempleo por departamento:
     DPTO grupo_edad         TD
0       5      18-24  17.685962
6       8      18-24  22.406918
12     11      18-24  17.797990
18     13      18-24  29.157386
25     15      24-30  14.491221
30     17      18-24   9.972584
36     18      18-24  21.686963
43     19      24-30   7.775308
48     20      18-24  19.042303
54     23      18-24  31.660884
60     25      18-24  15.618810
66     27      18-24  21.102675
72     41      18-24  14.602874
79     44      24-30  24.685637
84     47      18-24  20.675536
90     50      18-24  13.483416
96     52      18-24  16.529172
102    54      18-24  28.074930
109    63      24-30  14.237924
114    66      18-24  15.173556
120    68      18-24  23.222696
127    70      24-30  22.884184
132    73      18-24  19.348020
138    76      18-24  17.821703
144    81      18-24  61.095429
150    85      18-24  27.966955
156    86      18-24  44.878068
163    88      24-30  23.449477
171    91   

El desempleo se concentra principalmente en los jóvenes de 18 a 24 años, lo que refleja sus dificultades de inserción laboral. En algunos departamentos, como Norte de Santander, Valle del Cauca y Bolívar, los más afectados son los adultos jóvenes de 24 a 30 años. En contraste, la población de 60 años o más presenta las tasas más bajas, evidenciando una marcada brecha generacional.

4.¿Cuáles son los 3 departamentos con menor tasa de desempleo de mujeres entre los 18 y los 25 años ? 